# Motherscript.
The action to rule them all

In [1]:
import os
import pandas as pd
import numpy as np
from IPython import get_ipython

# envs, just in case
import dotenv
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting to check: ", notebook_path)
    


    # if there is no schedule, run immediately
    if schedule_in_days == None:
        EXECUTE = True
        


    # else run on a schedule based on last execution time
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = ["2002-07-16 18:32",np.nan]
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        import datetime
        now = datetime.datetime.now()
        try:
            last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        except:
            print('check log duplicates for issues here')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # check diff
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            




    # EXECUTE CRITERIA


    # EXECUTE TRUE
    if EXECUTE == True:
        print("Executtion: ", str(EXECUTE), "... STARTING NOW")

        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        # try to run script, if it fails, it fails
        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "... t'was a SUCCESS! :)"
            print(status)
        except:
            status = "... t'was a FAILURE. :("
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")




    # EXECUTE FALSE
    elif EXECUTE == False:
        print("Executtion: ", str(EXECUTE), "... DID NOT EXECUTE NOR LOG")



    print("Finished: ", notebook_path, "\n\n😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋\n\n")
    pass




########################################################
###################### SCHEDULE #######################
########################################################




# ##### BOLIGRAFICA #####
# SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_covid_replies/covid_reply.ipynb", schedule_in_days= 12/24)
# SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/SDSN/sdsn_map_post.ipynb", schedule_in_days=15/24)
# SCHEDULED_RUNNER(notebook_path = "BoliGrafica/_bots/twitter_bo-indicadores/Banco_Mundial_World_Bank/wbgapi_indicator_post.ipynb", schedule_in_days=14/24)

###### GREENHOUSE DATA #####
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb", schedule_in_days = 6/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/random_fig/random_fig.ipynb", schedule_in_days = 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb", schedule_in_days = 2)


print("🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻\n               FINITO\n🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻")

starting to check:  Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb
previous log found: 2023-02-03 20:16
difference in execution is: 0.6741 days
needed difference to execute: 1 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/_bots/twitter_country_post/country_of_the_day.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb
previous log found: 2023-02-03 20:16
difference in execution is: 0.6741 days
needed difference to execute: 0.25 days
Executtion:  True ... STARTING NOW
envs failed to load
Iran, Islamic Republic of greenhouse gas emission data and figures 👇

If you like this kind of stuff be sure to check out my website for many more datasets and figures! Thank you and have a wonderful day 🤖
Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Pierre and Miquelon"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting su

['country_data/CHE_Switzerland/figures/CHE_relative_totals.png',
 'country_data/CHE_Switzerland/figures/CHE_GCP_Country_Highlight.png',
 'country_data/CHE_Switzerland/figures/CHE_GCP_1.png',
 'country_data/CHE_Switzerland/figures/CHE_Minx_top20_subsectors.png']

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [False] 



Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Hong Kong"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Morocco"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Croatia"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Lebanon"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Lucia"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Malaysia"
 > found 1 tweets


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/tweepy/api.py:295: DeprecationWarning: invalid escape sequence \[
  """search_30_day(label, query, *, tag, fromDate, toDate, maxResults, \
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/tweepy/api.py:378: DeprecationWarning: invalid escape sequence \[
  """search_full_archive(label, query, *, tag, fromDate, toDate, \
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/tweepy/api.py:464: DeprecationWarning: invalid escape sequence \[
  """home_timeline(*, count, since_id, max_id, trim_user, \
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/tweepy/api.py:504: DeprecationWarning: invalid escape sequence \[
  """mentions_timeline(*, count, since_id, max_id, trim_user, \
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/tweepy/api.py:539: DeprecationWarning: invalid escape sequence \[
  """user_timeline(*, user_id, screen_name, since_id, count, max_id, \


Unauthorized: 401 Unauthorized
136 - You have been blocked from the author of this tweet.

... t'was a FAILURE. :(
New log entry successful!


Finished:  Greenhouse_Data/_bots/twitter_greenhouse_summon/greenhouse_summon.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/random_fig/random_fig.ipynb
previous log found: 2023-02-03 22:14
difference in execution is: 0.5924 days
needed difference to execute: 1 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/random_fig/random_fig.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


starting to check:  Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb
previous log found: 2023-02-03 22:14
difference in execution is: 0.5924 days
needed difference to execute: 2 days
Executtion:  False ... DID NOT EXECUTE NOR LOG
Finished:  Greenhouse_Data/NOAA_updates/NOAA_updates.ipynb 

😋😋😋😋😋😋😋😋 NEXT 😋😋😋😋😋😋😋😋😋


🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻
               FINITO
🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻🐻
